<a href="https://colab.research.google.com/github/gravity102424/ESAA/blob/main/ESAA_YB_week07_2_review_%EC%9C%A0%EC%A0%84%EC%B2%B4_%EC%A0%95%EB%B3%B4_%ED%92%88%EC%A2%85_%EB%B6%84%EB%A5%98_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 프로젝트 개요

* 대회 주제: SNP(유전자형) 기반 다중 분류 문제

* 핵심 목표:
데이터 불균형과 범주형 SNP 변수의 높은 차원성을 해결하면서
예측 정확도를 높이는 분류 모델 구축

* 팀 전략:

  ① 도메인 특화 피처 엔지니어링

  ② CatBoostEncoder 기반 타깃 인코딩

  ③ BorderlineSMOTE를 활용한 불균형 보정

  ④ 다양한 분류기 조합의 가중치 보팅

# 2. Dataset Info.

train.csv [파일]

* id : 개체 고유 ID
* 개체정보
  * father : 개체의 가계 고유 번호 (0 : Unknown)
  * mother : 개체의 모계 고유 번호 (0 : Unknown)
  * gender : 개체 성별 (0 : Unknown, 1 : female, 2 : male)
  * trait : 개체 표현형 정보
* 15개의 SNP 정보 : SNP_01 ~ SNP_15
* class : 개체의 품종 (A,B,C)


test.csv [파일]
* id : 개체 샘플 별 고유 ID
* 개체정보
  * father : 개체의 가계 고유 번호 (0 : Unknown)
  * mother : 개체의 모계 고유 번호 (0 : Unknown)
  * gender : 개체 성별 (0 : Unknown, 1 : female, 2 : male)
  * trait : 개체 표현형 정보
* 15개의 SNP 정보 : SNP_01 ~ SNP_15


snp_info.csv [파일]
* 15개의 SNP 세부 정보
  * name : SNP 명
  * chrom : 염색체 정보
  * cm : Genetic distance
  * pos : 각 마커의 유전체상 위치 정보


sample_submission.csv [파일] - 제출 양식
* id : 개체 샘플 별 고유 ID
* class : 예측한 개체의 품종 (A,B,C)


# 3. 데이터 전처리 및 피처 엔지니어링
## (1) 기본 처리

* id, class 분리 후 학습용(train_x, train_y), 테스트(test_x) 구성

* 결측치 없음, 문자열 기반 SNP feature 다수 존재

## (2) 피처 생성

* SNP 문자열 결합 피처

  * 예: chrom_6, 6_ARS_BFGL, 9_HAPMAP, 10_BOVINE

  * 여러 SNP 값을 문자열로 연결해 염색체/유전자 단위 패턴을 반영

* 염기 카운트 기반 수치 피처

  * 'A', 'G', 'C' 등장 횟수 계산

  * numGC, numA, numGC², sub, H = numGC*3 + numA*2 생성
    
    → 유전자의 조성 특징을 반영

## (3) 데이터 타입 분리 및 스케일링

* 범주형(object) vs 수치형 분리

* **StandardScaler()**로 수치형만 스케일링 (데이터 누수 방지)


# 4. 인코딩 기법: CatBoostEncoder (다중 클래스 확장)
## (1) 기본 원리

* 각 범주형 값이 목표변수(class)와 얼마나 밀접한지를 수치화

  → 타깃 통계치(mean target probability)를 반영

## (2) 구현 방식

* 다중 클래스 문제를 클래스별 이진화(One-vs-Rest)

* 각 클래스에 대해 CatBoostEncoder() 적용 후
인코딩된 결과를 열 단위로 병합

* 결과: 클래스별로 의미 있는 통계량이 반영된 확장형 피처셋

# 5. 클래스 불균형 처리

* BorderlineSMOTE(random_state=26) 사용

* 단순 오버샘플링이 아닌, 결정 경계 근처 샘플만 증강
  
  → 오버피팅 방지 및 경계 학습 강화

# 6. 모델링: VotingClassifier 앙상블
## (1) 개별 모델 구성
| 분류기                        | 핵심 특징        |
| :------------------------- | :----------- |
| BaggingClassifier          | 고분산 완화       |
| DecisionTreeClassifier     | 비선형 경계 탐지    |
| RidgeClassifier / RidgeCV  | 선형 안정성       |
| XGBClassifier              | 부스팅 기반 강력 모델 |
| LGBMClassifier             | 빠르고 효율적인 부스팅 |
| GradientBoostingClassifier | 단계적 향상       |
| SVC                        | 마진 최대화       |
| RandomForestClassifier     | 앙상블 안정성      |

## (2) 하드보팅 구성

* VotingClassifier(voting='hard')
* 가중치: [1,1,1,2,1,1,1,1,2]
  
  → XGB와 RF의 영향력 강화
* 학습: fit(train_x, train_y)
* 예측: inverse_transform(best_model.predict(test_x))

# 7. 결과 및 성능

* 강력한 모델 조합 및 인코딩 전략으로
* 과적합 없이 높은 정확도 확보

CatBoostEncoder의 효과로 범주형 정보 손실 최소화

# 8. 차별점 및 배울 점

| 구분            | 내용                              |
| :------------ | :------------------------------ |
| **인코딩 혁신성**   | CatBoostEncoder를 멀티클래스용으로 직접 확장 |
| **도메인 피처 설계** | SNP 문자열 구조를 반영한 결합 피처           |
| **데이터 증강 전략** | BorderlineSMOTE로 의미 있는 증강       |
| **앙상블 최적화**   | XGB·RF 중심의 가중 보팅 구성             |
